## PDI - Phuture DeFi Index (Multi-chain) - Methodology Template

## Configuring packages

In [9]:
import pandas as pd
import decouple
import sys
sys.path.append('../')
from methodology import MethodologyBase
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

## Index Variables

In [10]:
min_mcap =  2.5e8
min_weight = 0.005
max_weight = 0.3
circ_supply_threshold = 0.2
liveness_threshold = 180 # In days
liquidity_consistency = 90 # In days
coingecko_category = "decentralized-finance-defi"

# Calculating max slippage based on liquidity threshold
liq = 5*1e6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .01
max_slippage = round(max_slippage, 2)

# Dataframes to remove
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')

methodology_class = MethodologyBase(
                                min_mcap,
                                min_weight,
                                max_weight,
                                255,
                                circ_supply_threshold,
                                liveness_threshold,
                                liquidity_consistency,
                                max_slippage,
                                coingecko_category)

## Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data spanning at least 6 months prior to the date of inclusion in the index.
    1. The project should have an active community.
    1. No rebasing or deflationary tokens.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project must have a minimum circulating market cap of $50mm.
   <br>
1. **Pricing requirmeents**
    1. Token should have a reliable price feed from one of our authorised providers. 
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 20% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The price should experience no more than 12% price impact when executing a $100k trade.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.
1. **Weighting requirements**
    1. The maximum weight any one token can have is 30%.
    1. All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
    1. Any asset with a weight below 0.5% will be removed from the index.


## Run Methodology

In [11]:

results,liquidity = methodology_class.main(
    df_to_remove=[stablecoins,ctokens,atokens],
    remove_category_assets=['the-graph','staked-ether','injective-protocol']
    )
liquidity

0xbec243c995409e6520d7c41e404da5deba4b209b avalanche
0xa0b862f60edef4452f25b4160f177db44deb6cf1 arbitrum-one
0x7205705771547cf79201111b4bd8aaf29467b9ec polygon-pos
0x9046d36440290ffde54fe0dd84db8b1cfee9107b optimistic-ethereum
0x9eaf8c1e34f05a589eda6bafdf391cf6ad3cb239 base
0xd7c49cee7e9188cca6ad8ff264c1da2e69d4cf3b ethereum


,spot price,delivery price,slippage,blockchain
id,,,,
chainlink,0.058178,5.805914e-02,-0.002047,ethereum
convex-finance,0.236956,2.363567e-01,-0.002528,ethereum
pancakeswap-token,0.393146,3.914479e-01,-0.004319,binance-smart-chain
ribbon-finance,2.892401,2.878811e+00,-0.004699,ethereum
gmx,0.018438,1.834250e-02,-0.005175,arbitrum-one
rocket-pool,0.031803,3.160325e-02,-0.006277,ethereum
lido-dao,0.405902,4.032983e-01,-0.006415,ethereum
uniswap,0.151848,1.505836e-01,-0.008329,arbitrum-one
havven,0.256775,2.538344e-01,-0.011451,optimistic-ethereum


## Show Results

In [12]:
results

,name,market_cap,weight,weight_converted,address,blockchain_with_highest_liq
id,,,,,,
chainlink,Chainlink,9546046048,0.300000,77,0x514910771af9ca656af840dff83e8264ecf986ca,ethereum
uniswap,Uniswap,4961978532,0.204447,53,0xfa7f8980b0f1e64a2062791cc3b0871572f1f7f0,arbitrum-one
lido-dao,Lido DAO,2189523118,0.090214,24,0x5a98fcbea516cf06857215779fd812ca3bef1b32,ethereum
aave,Aave,1470470920,0.060587,16,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,ethereum
maker,Maker,1334847596,0.054999,15,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2,ethereum
havven,Synthetix Network,1278188733,0.052665,14,0x8700daec35af8ff88c16bdf0418774cb3d7599b4,optimistic-ethereum
frax-share,Frax Share,674463057,0.027790,8,0x3432b6a60d23ca0dfca7761b7ab56459d9c964d0,ethereum
pancakeswap-token,PancakeSwap,660144151,0.027200,6,0x0e09fabb73bd3ade0a17ecc321fd13a19e81ce82,binance-smart-chain
curve-dao-token,Curve DAO,642990339,0.026493,6,0xd533a949740bb3306d119cc777fa900ba034cd52,ethereum
